In [ ]:
000000000000000000000000000000000!21zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzaz A  QQQQQQQQQQQQQ                                                                                                                                                                                                                                                                                                                                                   0pip install transformers==4.2.  2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 20.9 MB/s 
     |████████████████████████████████| 880 kB 55.4 MB/s 
     |████████████████████████████████| 2.9 MB 61.2 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=396327157b6ca1caa60efdef449cea55e3ff661a47bbaf19415d725a3df5201e
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [ ]:
!nvidia-smi

Thu Jul 21 18:05:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Load the dataset




In [ ]:
! gdown 1g62U31BbbTSudCAk46l2tVSUhoa4_5Ti
! gdown 1azLzw9kHF0mtNF7LtzRUFc-PousKeTqr

Downloading...
From: https://drive.google.com/uc?id=1g62U31BbbTSudCAk46l2tVSUhoa4_5Ti
To: /content/test_Sherlock.txt
100% 58.3k/58.3k [00:00<00:00, 50.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1azLzw9kHF0mtNF7LtzRUFc-PousKeTqr
To: /content/train_Sherlock.txt
100% 537k/537k [00:00<00:00, 142MB/s]


In [ ]:
with open('train_Sherlock.txt', 'r') as f:
    train_data = f.read()

with open('test_Sherlock.txt', 'r') as f:
    test_data = f.read()

In [ ]:
print("Train dataset length: "+str(len(train_data)))
print("Test dataset length: "+ str(len(test_data)))

Train dataset length: 524763
Test dataset length: 57103


#Preprocess

In [ ]:
import string


def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation+"\”"])
    return punctuationfree

In [ ]:
def preprocess(text):
    text = remove_punctuation(text)
    text = ' '.join(text.split())
    text = text.lower()
    return text

In [ ]:
train_data = preprocess(train_data)
test_data = preprocess(test_data)

In [ ]:
with open('train_Sherlock_preprocessed.txt', 'w') as f:
    f.write(train_data)
with open('test_Sherlock_preprocessed.txt', 'w') as f:
    f.write(test_data)

# Build a TextDataset


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
from transformers import TextDataset,DataCollatorForLanguageModeling


train_path = 'train_Sherlock_preprocessed.txt'
test_path = 'test_Sherlock_preprocessed.txt'

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,
Token indices sequence length is longer than the specified maximum sequence length for this model (104759 > 1024). Running this sequence through the model will result in indexing errors


# Initialize Trainer and GPT-2 model

In [ ]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("distilgpt2", pad_token_id=tokenizer.eos_token_id)


training_args = TrainingArguments(
    output_dir="./gpt2", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=5, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:925: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/353M [00:00<?, ?B/s]

In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=130, training_loss=4.790209021935096, metrics={'train_runtime': 92.9845, 'train_samples_per_second': 1.398, 'total_flos': 257297230725120, 'epoch': 5.0})

Save the model.

In [ ]:
trainer.save_model()

In [ ]:
model = AutoModelWithLMHead.from_pretrained("./gpt2", pad_token_id=tokenizer.eos_token_id)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:925: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
model.config

GPT2Config {
  "_name_or_path": "./gpt2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 6,
  "n_positions": 1024,
  "pad_token_id": 50256,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.2.2",
  "use_cache": true,
  "vocab_size": 50257
}

# Demo text generation with different decoding strategies

Text-generation pipeline

In [ ]:
from transformers import pipeline

generator = pipeline('text-generation',model='./gpt2', tokenizer='gpt2',config={'max_length':200})


##Greedy

In [ ]:
model = AutoModelWithLMHead.from_pretrained("./gpt2", pad_token_id=tokenizer.eos_token_id)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:925: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
input_ids = tokenizer.encode('I could', return_tensors='pt')

# generate text until the output length (which includes the context length) reaches 20
greedy_output = model.generate(input_ids, max_length=20)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I could not be more pleased with the fact that the man who had been in the room with me


##Beam

In [ ]:
beam_output = model.generate(
    input_ids, 
    max_length=20, 
    num_beams=5, 
)
print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1632: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


Output:
----------------------------------------------------------------------------------------------------
I could not be more pleased with the fact that I had been in a position of power for a


##Sampling

In [ ]:
# activate sampling and deactivate top_k by setting top_k sampling to 0
sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=20, 
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I could not sleep enough to post something to this article without very believing that the letter had gone through


##Sampling with Temperature

In [ ]:
# use temperature to decrease the sensitivity to low probability candidates
temperature_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=20, 
    top_k=0, 
    temperature=0.9
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(temperature_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I could feel the uneasiness of a man stirring against them or else shaking his hands and feeling


##Top-K

In [ ]:
# set top_k to 40
topk_40_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=20, 
    top_k=40
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(topk_40_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I could hear and I had been a little nervous but my feet were getting better from the sound of


##Nucleus

In [ ]:
# sample only from 90% most likely words
nucleus_p090_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=20, 
    top_p=0.9, 
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(nucleus_p090_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I could have sworn it off myself as one of the great instructors I had sent to school, the


In [ ]:
# sample only from 95% most likely words
nucleus_p095_output = model.generate(
    input_ids,
    do_sample=True, 
    max_length=20, 
    top_p=0.95, 
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(nucleus_p095_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I could not resist my glimmering glance at the temple and a glance at the emerald-


#Evaluation

## Count repeated percentage

In [ ]:
import nltk, re
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
import torch
from torch.utils.data import Dataset, DataLoader

class PromptDataset(torch.utils.data.Dataset):
    def __init__(self, text_path, prompt_length):
        self.prompts = self.load_sentences(text_path, prompt_length)
        
    def load_sentences(self, text_path, prompt_length=2):
        prompts = []
        with open(text_path, 'r') as f:
            text = f.read()
        # Split text into sentences
        sentences = sent_tokenize(text.strip())
        for sent in sentences[1:201]:
            tokenized_sent = word_tokenize(sent.lower())
            remove_punct = [word for word in tokenized_sent if re.search(r'\w+', word) is not None]
            prompt = ' '.join(remove_punct[:prompt_length])
            real = ' '.join(remove_punct)
            prompts.append((prompt, real))
        return prompts
    
    def __len__(self):
        return len(self.prompts)
    
    def __getitem__(self, idx):
        return self.prompts[idx]


test_data = PromptDataset('test_Sherlock.txt', 2)
test_loader = DataLoader(test_data, batch_size=1, shuffle=False)


from collections import Counter

def count_repetition_percent(generated_strings):
    words = generated_strings.split()
    counter = Counter(words)
    count_repeated = 0
    for word in counter:
        count_repeated += (counter[word] - 1)
    sentence_length = sum(counter.values())
    return count_repeated / sentence_length

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
decode_strats = {
   'greedy' : {},
   'beam 3' : {'num_beams':3},
   'beam 5': {'num_beams':5},
   'beam 7': {'num_beams':7},
   'random sampling': {'do_sample':True, 'top_k':0},
   'temperature 0.5': {'do_sample':True, 'top_k':0, 'temperature':0.5},
   'temperature 0.7': {'do_sample':True, 'top_k':0, 'temperature':0.7},
   'temperature 0.9': {'do_sample':True, 'top_k':0, 'temperature':0.9},
   'top 5': {'do_sample':True, 'top_k':5},
   'top 20': {'do_sample':True, 'top_k':20},
   'top 40': {'do_sample':True, 'top_k':40},
   'nucleus p = 0.8': {'do_sample':True, 'top_k':0, 'top_p':0.8},
   'nucleus p = 0.9': {'do_sample':True, 'top_k':0, 'top_p':0.9},
   'nucleus p = 0.95': {'do_sample':True, 'top_k':0, 'top_p':0.95},
  }

In [ ]:
for strat in decode_strats.keys():
    print('-'*100)
    print('Decode strat:', strat)
    print('Params:',decode_strats[strat] )
    results = []
    for s in test_loader:
        input_ids = tokenizer.encode(s[0][0], return_tensors='pt')
        sample_output = model.generate(
        input_ids, 
        **decode_strats[strat],
        max_length = 20
        )
        generated_strings = tokenizer.decode(sample_output[0], skip_special_tokens=True)
        results.append(count_repetition_percent(generated_strings) )
    print('Average count_repetition_percent:',sum(results) / len(results))

----------------------------------------------------------------------------------------------------
Decode strat: greedy
Params: {}
Average count_repetition_percent: 0.4510735178349345
----------------------------------------------------------------------------------------------------
Decode strat: beam 3
Params: {'num_beams': 3}


/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1632: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


Average count_repetition_percent: 0.3576914252543122
----------------------------------------------------------------------------------------------------
Decode strat: beam 5
Params: {'num_beams': 5}
Average count_repetition_percent: 0.34821628736760313
----------------------------------------------------------------------------------------------------
Decode strat: beam 7
Params: {'num_beams': 7}
Average count_repetition_percent: 0.3502813889327048
----------------------------------------------------------------------------------------------------
Decode strat: random sampling
Params: {'do_sample': True, 'top_k': 0}
Average count_repetition_percent: 0.0833816602214434
----------------------------------------------------------------------------------------------------
Decode strat: temperature 0.5
Params: {'do_sample': True, 'top_k': 0, 'temperature': 0.5}
Average count_repetition_percent: 0.2682730877438694
------------------------------------------------------------------------------